# Title

In [1]:
#imports
import pandas as pd
import numpy as np
import acquire as a
import prepare as p
import explore as e
import model_functions as m 


# Acquire

In [2]:
df = a.get_heart()

# Prepare

<div class = "alert alert-block block alert-info">
    <b> testing if this changes color  </b>
</div>

Dropped rows with nulls (only 2 rows)

In [3]:
#renaming columns to make them more readable
df = p.rename_cols(df)
# replacing integer values with their corresponding strings for discrete variables
df = p.replace_cat_values(df)
# split to train, validate and test
train, val, test = p.split_data(df, 'high_risk_of_mi')

# Explore

In [ ]:
#explore on train data only

# Exploration Summary

# Modeling

In [4]:
df = m.df_classification_ready(df, 'sex_female')


In [ ]:
df.info()

In [5]:
#train, val, test = p.split_data(df, 'high_risk_of_mi')

X_Train, y_Train, X_val, y_val, X_test, y_test = m.isolate_target(train, val, test, 'high_risk_of_mi')

#### Best 3 Models

#### Test Model

# Modeling Wrap

# Conclusion

#### Summary

#### Recommendations

#### Next Steps